Initially I forked from this [kernel](https://www.kaggle.com/khursani8/fast-ai-starter-resnet34), changed architecture to ResNet 50, added augmentation and did some initial tuning of parameters like learning rate.

# Libraries import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import fastai
from fastai.vision import *
fastai.__version__

In [ ]:
# copy pretrained weights for resnet50 to the folder fastai will search by default
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp /kaggle/input/fastai-pretrained-models/* /tmp/.cache/torch/checkpoints/

# Data

In [ ]:
PATH = Path('../input/aptos2019-blindness-detection')

In [ ]:
df = pd.read_csv(PATH/'train.csv')
df.head()

In [ ]:
!ls /kaggle/input/fastai-pretrained-models/

In [ ]:
df.diagnosis.value_counts() 

So our train set is definitely imbalanced, majority of images are normal (without illness).

# Model

In [ ]:
src = (
    ImageList.from_df(df,PATH,folder='train_images',suffix='.png')
        .split_by_rand_pct(0.1, seed=42)
        .label_from_df()
    )
src

In [ ]:
tfms = get_transforms(do_flip=True, 
                      flip_vert=True, 
                      max_rotate=360, 
                      max_zoom=1.2, 
                      max_warp=0.2, 
                      max_lighting=0.2,
                     xtra_tfms=[(symmetric_warp(magnitude=(-0,0), p=0)),])

In [ ]:
data = (
    src.transform(tfms,size=64)
    .databunch(bs=64)
    .normalize(imagenet_stats)
)
data

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()

    
def roc_score(inp, target):
    _, indices = inp.max(1)
    return torch.Tensor([roc_auc_score(target, indices)])[0]

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"
loss_func = FocalLoss()
learn = cnn_learner(data,models.densenet201 ,metrics=[accuracy,kappa],loss_func=loss_func,model_dir='/kaggle/working')

In [ ]:
# Find a good learning rate
learn.lr_find()
learn.recorder.plot()

In [ ]:
!nvidia-smi

In [ ]:
lr = 1e-2
learn.fit_one_cycle(2, slice(lr))

In [ ]:
learn.save('epoch-02')

In [ ]:
lr = 1e-2
learn.fit_one_cycle(2, slice(lr))

In [ ]:
# lr = 1e-2
# learn.fit_one_cycle(2, slice(lr))

In [ ]:
learn.save('epoch-04')

In [ ]:
learn.load('epoch-04')

In [ ]:
# progressive resizing
learn.data = data = (
    src.transform(tfms,size=128)
    .databunch(bs=64)
    .normalize(imagenet_stats)
)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(2, slice(lr))

In [ ]:
# lr = 1e-3
# learn.fit_one_cycle(2, slice(lr))

In [ ]:
learn.save('epoch-22')

In [ ]:
learn.load('epoch-22')

In [ ]:
# # progressive resizing
# learn.data = data = (
#     src.transform(tfms,size=224)
#     .databunch(bs=4)
#     .normalize(imagenet_stats)
# )

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# lr = 1e-3
# learn.fit_one_cycle(10, slice(lr))

In [ ]:
# learn.save('epoch-30')

In [ ]:
# # progressive resizing
# learn.data = data = (
#     src.transform(tfms,size=384)
#     .databunch(bs=1)
#     .normalize(imagenet_stats)
# )

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# learn.unfreeze()

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# learn.fit_one_cycle(10, slice(1e-5,1e-3))

# Predictions

In [ ]:
# test_df = pd.read_csv(PATH/'test.csv')
# test_df.head()
sample_df = pd.read_csv(PATH/'sample_submission.csv')
sample_df.head()

In [ ]:
learn.data.add_test(ImageList.from_df(sample_df,PATH,folder='test_images',suffix='.png'))

In [ ]:
preds,y = learn.get_preds(DatasetType.Test)

In [ ]:
sample_df.diagnosis = preds.argmax(1)
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv',index=False)

In [ ]:
torch.save(learn.model[0].state_dict(),'/kaggle/working/aptosdensenet201.h5')

In [ ]:
learn.model